In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from nbc_analysis.utils.config_utils import get_config
from nbc_analysis.extracts.main import main as run_extracts
from nbc_analysis.utils.debug_utils import runit

In [11]:
runit(get_config)

created example config file at: /Users/wmcabee/.config/nbc_analysis/extracts.yaml


{'DAYS': ['20190701',
  '20190702',
  '20190703',
  '20190704',
  '20190705',
  '20190706',
  '20190707',
  '20190708',
  '20190709',
  '20190710',
  '20190711',
  '20190712',
  '20190713',
  '20190714',
  '20190715',
  '20190716',
  '20190717',
  '20190718',
  '20190719',
  '20190720',
  '20190721',
  '20190722',
  '20190723',
  '20190724',
  '20190725',
  '20190726',
  '20190727',
  '20190728',
  '20190729',
  '20190730',
  '20190731'],
 'EVENT_SET_D': '/Users/wmcabee/DATA/NBC2/event_set',
 'EXTRACT_SPECS': {'android': {'prefix': 'NBCProd/Android/NBC_{day}'}},
 'LIMIT': 3000,
 'RAW_EVENTS_BUCKET': 'nbc-digital-cloned',
 'WORK_D': '/Users/wmcabee/DATA/NBC2/tmp'}

In [12]:
ret = runit(run_extracts)
ret

>> init extract run
created example config file at: /Users/wmcabee/.config/nbc_analysis/extracts.yaml
>> start extract run, config={'DAYS': ['20190701', '20190702', '20190703', '20190704', '20190705', '20190706', '20190707', '20190708', '20190709', '20190710', '20190711', '20190712', '20190713', '20190714', '20190715', '20190716', '20190717', '20190718', '20190719', '20190720', '20190721', '20190722', '20190723', '20190724', '20190725', '20190726', '20190727', '20190728', '20190729', '20190730', '20190731'], 'EVENT_SET_D': '/Users/wmcabee/DATA/NBC2/event_set', 'EXTRACT_SPECS': {'android': {'prefix': 'NBCProd/Android/NBC_{day}'}}, 'LIMIT': 3000, 'RAW_EVENTS_BUCKET': 'nbc-digital-cloned', 'WORK_D': '/Users/wmcabee/DATA/NBC2/tmp'}
>> start extract,extract=android,outfile=/Users/wmcabee/DATA/NBC2/event_set/android_20190701.csv
>> end extract,extract=android,outfile=/Users/wmcabee/DATA/NBC2/event_set/android_20190701.csv,records=1717
>> start extract,extract=android,outfile=/Users/wmcabee/D